### Exploring Pytorch, JAX, and Tensorflow

Cite like this [[1]](#ref-1), and this [[2]](#ref-2). Our notebook is available on github at [[3]](#ref-3).

### References
<!-- use two extra spaces at end of each line for line break -->
[1] <a id="ref-1"></a> [Andrej Karpathy Lecture](https://www.youtube.com/watch?v=VMj-3S1tku0)  
[2] <a id="ref-2"></a> [Micrograd Repository](https://github.com/karpathy/micrograd)  
[3] <a id="ref-3"></a> [Our Jupyter notebook in this post](https://github.com/sriramgkn/micrograd-sri)  
